In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# DecisionTreeClassifier

In [21]:
#load and encode train data
df=pd.read_csv("../data/train.csv")
X=df[["Sex","Pclass","Embarked","Age"]]
y=df[["Survived"]]
encoder = LabelEncoder()
for col in ['Sex', 'Embarked', "Age"]:
    X[col] = encoder.fit_transform(X[col])

#fit model
clf=DecisionTreeClassifier()
clf.fit(X,y)

#load test X
df_test_X=pd.read_csv("../data/test.csv")
X_test=df_test_X[["Sex","Pclass","Embarked","Age"]]
encoder = LabelEncoder()
for col in ['Sex', 'Embarked', "Age"]:
    X_test[col] = encoder.fit_transform(X_test[col])

#make prediction
y_pred=clf.predict(X_test)

#load test y and ensure that this dataset maches test x
df_test_y=pd.read_csv("../data/didsurvive.csv")
# print(df_test_X, df_test_X.shape)
# print(df_test_y, df_test_y.shape)
y_test=df_test_y["Survived"]

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

0.7200956937799043


/tmp/ipykernel_43476/1222180252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = encoder.fit_transform(X[col])
/tmp/ipykernel_43476/1222180252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = encoder.fit_transform(X[col])
/tmp/ipykernel_43476/1222180252.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [24]:
# X, y - train X and y
# X_test, y_test - test X and y

MAX_DEPTH=4

#make new model and fit it
clf2=DecisionTreeClassifier(max_depth=MAX_DEPTH)
clf2.fit(X,y)

#make prediction
y_pred=clf2.predict(X_test)

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

0.8923444976076556


In [36]:
df_test=pd.concat([X_test, y_test], axis=1)
df_test

,Sex,Pclass,Embarked,Age,Survived
0,1,3,1,44,0
1,0,3,2,60,1
2,1,2,1,74,0
3,1,3,2,34,0
4,0,3,2,27,1
...,...,...,...,...,...
413,1,3,2,79,0
414,0,1,0,51,1
415,1,3,2,50,0
416,1,3,2,79,0


In [42]:
df_men=df_test[df_test["Sex"]==1]
print(df_men["Survived"].unique())
df_women=df_test[df_test["Sex"]==0]
print(df_women["Survived"].unique())

[0]
[1]


In this case in test dataset all woman have survived and all man died so the best depth is 1 with accuracy = 1. With higher depth model is overfitted.